### 필터링 기준 (병원)
> 사용 열 : "상세영업상태명", "폐업일자", "소재지전체주소", "도로명전체주소", "사업장명", "업태구분명", "의료인수", "입원실수", "병상수", "진료과목내용명"

- 필터링 (1) : 분당, 일산 단어가 포함된 위치 우선적으로 선택
- 전처리 (1) : 주소 기반 위도, 경도 관련 데이터 Search

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/shinjh/Desktop/likelion/mid_project/dataset_life_infra")

In [ ]:
## 함수 init (결측 위경도 추가용)
import pandas as pd
import requests
import json
import re
from tqdm import tqdm
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="cocoa")

# Kakao API로 좌표 검색 함수
def get_location_kakao(address_data, kakao_api_key):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address_data}'
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        api_json = response.json()
        if api_json['documents']:
            address = api_json['documents'][0]['address']
            return float(address['y']), float(address['x'])  # 위도, 경도 반환
    return None, None

# Nominatim으로 좌표 검색 함수
def get_location_nominatim(address_data):
    result = clean_address(address_data)
    # print(result)
    location = geolocator.geocode(result)
    if location:
        return location.latitude, location.longitude  # 위도, 경도 반환
    return None, None

def clean_address(address):
    # 도로명 기준으로 작동하기에, 첫번째로 나오는 '로' 촥인
    # Step 1: '번길'과 괄호 내용 제거
    step_1 = re.sub(r'\s*번길|\(.*\)', '', address)

    # Step 2: '층' 관련 정보 제거
    step_2 = re.sub(r'\s*[\d~]+층', '', step_1)

    # Step 3: '호' 관련 정보 제거
    step_3 = re.sub(r'[\d~]*호', '', step_2)

    # Step 4: 콤마 뒤 내용 제거
    cleaned = re.sub(r',.*', '', step_3)

    return cleaned

# 데이터프레임 업데이트 함수
def update_coordinates(df, kakao_api_key):
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            # 1단계: Kakao API로 '소재지전체주소' 검색 시도
            address = row['소재지전체주소']
            lat, lon = get_location_kakao(address, kakao_api_key)
            
            if lat is None or lon is None:
                raise ValueError("Latitude or Longitude is None in Kakao API (소재지전체주소)")
        
        except:
            try:
                # 2단계: Kakao API로 '도로명전체주소' 검색 시도
                address = row['도로명전체주소']
                lat, lon = get_location_kakao(address, kakao_api_key)
                
                if lat is None or lon is None:
                    address = row['소재지전체주소']

                    # ## 일산 케이스                    
                    # # 특이 케이스로 보이는 외*필지, 
                    # address = re.sub(r'외\d*필지.*|외 \d*필지.*', '', address)
                    # address = re.sub(r'번지.*', '', address)
                    # address = re.sub(r'\..*', '', address)
                    
                    # ## 분당 케이스                    
                    # # 특이 케이스로 보이는 외*필지, 
                    # address = re.sub(r'(\D동)(\d)', r'\1 \2', address)
                    # address = re.sub(r'외\d*필지.*|외 \d*필지.*', '', address)
                    # address = re.sub(r'번지.*', '', address)
                    # address = re.sub(r'\..*', '', address)
 
                    lat, lon = get_location_kakao(address, kakao_api_key)
                    
                    if lat is None or lon is None:
                        lat, lon = get_location_nominatim(address)
                
                # try쪽에서 아무것도 안될경우,
                if lat is None or lon is None:     
                    raise ValueError("Latitude or Longitude is None in Kakao API (소재지전체주소)")
                        
            except:
                try:
                    # 3단계: Nominatim으로 clean_address('도로명전체주소') 검색 시도
                    clean_addr = clean_address(row['도로명전체주소'])
                    lat, lon = get_location_nominatim(clean_addr)
                    
                    if lat is None or lon is None:
                        # 4단계: '도로명전체주소'에서 도로명과 숫자를 추출해 검색 시도
                        match = re.search(r"(.*로)(\d+)", clean_addr)
                        if match:
                            road_name = match.group(1)  # 도로명 (예: "호수로")
                            number = match.group(2)     # 숫자 (예: "817")
                            formatted_address = f"{road_name} {number}"  # 띄어쓰기 추가
                            # print(f"Nominatim with formatted address: {formatted_address}")
                            lat, lon = get_location_nominatim(formatted_address)
                            
                    if lat is None or lon is None:
                        # 4단계: '도로명전체주소'에서 도로명과 숫자를 추출해 검색 시도
                        match = re.search(r"(.*로 \d+)", clean_addr)
                        if match:
                            road_name = match.group(1)  # 도로명 (예: "호수로")
                            number = match.group(2)     # 숫자 (예: "817")
                            formatted_address = f"{road_name} {number}"  # 띄어쓰기 추가
                            # print(f"Nominatim with formatted address: {formatted_address}")
                            lat, lon = get_location_nominatim(formatted_address)
                            
                    # try쪽에서 아무것도 안될경우,
                    if lat is None or lon is None:     
                        raise ValueError("Latitude or Longitude is None in Kakao API (소재지전체주소)")
                    
                except:
                    # 모든 시도가 실패하면 continue
                    print(f"Failed to get location info for: {row['소재지전체주소']} / {row['도로명전체주소']}")
                    continue

        # 결과 저장
        df.at[idx, '위도'] = lat
        df.at[idx, '경도'] = lon           

# Kakao API Key 설정
kakao_api_key = "개인KEY"  # 카카오 API 키를 여기에 입력

In [102]:
## 병원
data_hospital = pd.read_csv("./병원.csv", encoding='utf-8')
data_hospital_sort = data_hospital[["상세영업상태명", "인허가일자","폐업일자", "소재지전체주소", "도로명전체주소", "사업장명", "업태구분명", "의료인수", "입원실수", "병상수", "진료과목내용명"]]

In [83]:
data_hospital_sort = data_hospital_sort.copy()
data_hospital_sort['위도'] = None
data_hospital_sort['경도'] = None

In [ ]:
ilsan = data_hospital_sort[
    data_hospital_sort["도로명전체주소"].combine_first(data_hospital_sort["소재지전체주소"]).str.contains("고양시 일산", na=False)
].reset_index(drop=True)

In [ ]:
update_coordinates(ilsan, kakao_api_key)
ilsan.to_csv("./dataset_life_infra_일산_병원.csv", encoding='utf-8')

In [99]:
ilsan.head()

,상세영업상태명,인허가일자,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,의료인수,입원실수,병상수,진료과목내용명,위도,경도
0,영업중,2007-01-24,NaN,경기도 고양시 일산서구 주엽동 21 사과나무치과병원,"경기도 고양시 일산서구 중앙로 1450, 사과나무치과병원 2~7층 (주엽동)",사과나무치과병원,치과병원,25,0,0,"내과, 가정의학과, 구강악안면외과, 치과보철과, 치과교정과, 소아치과, 치주과, 치...",37.671268,126.759604
1,영업중,2008-07-11,NaN,"경기도 고양시 일산동구 중산동 1696 지하1층 일부, 1층 일부, 2~5층","경기도 고양시 일산동구 중산로 117, 지하1층 일부, 1층 일부, 2~5층 (중산동)",해븐리병원,병원,37,40,178,"내과, 신경과, 정형외과, 영상의학과, 재활의학과, 가정의학과",37.682541,126.781049
2,영업중,2008-08-14,NaN,NaN,"경기도 고양시 일산서구 주화로 88 (주엽동, 효림빌딩 201호, 205~6호, 3...",효림요양병원,요양병원(일반요양병원),20,29,150,"내과, 신경과, 외과, 피부과, 재활의학과, 가정의학과, 한방내과, 한방신경정신과,...",37.670554,126.758525
3,영업중,2009-03-31,NaN,NaN,"경기도 고양시 일산서구 덕이로 212 (덕이동, 백송프라자 3,4층 일부)",무지개요양병원,요양병원(일반요양병원),41,49,280,"가정의학과, 한방내과",37.696692,126.739335
4,영업중,2009-04-29,NaN,"경기도 고양시 일산서구 일산2동 554번지 4호 일산마샬빌딩 4,5,6,7,8층","경기도 고양시 일산서구 일중로 59 (일산동, 일산마샬빌딩 4,5,6,7,8층)",로하스일산병원,병원,48,31,112,"내과, 재활의학과, 가정의학과, 한방내과",37.684972,126.775199


In [92]:
bundang = data_hospital_sort[
    data_hospital_sort["도로명전체주소"].combine_first(data_hospital_sort["소재지전체주소"]).str.contains("성남시 분당", na=False)
].reset_index(drop=True)

In [94]:
update_coordinates(bundang, kakao_api_key)

  4%|▍         | 2/46 [00:00<00:02, 16.71it/s]

경기도 성남시 분당구 황새울로 8
경기도 성남시 분당구 미금일로 32


  9%|▊         | 4/46 [00:01<00:20,  2.01it/s]

경기도 성남시 분당구 황새울로 15


 13%|█▎        | 6/46 [00:02<00:16,  2.42it/s]

...

 96%|█████████▌| 44/46 [00:14<00:00,  3.43it/s]

경기도 성남시 분당구 정자일로 6


 98%|█████████▊| 45/46 [00:15<00:00,  2.82it/s]

경기도 성남시 분당구 장미로 78


100%|██████████| 46/46 [00:15<00:00,  2.94it/s]


In [97]:
bundang.to_csv("./dataset_life_infra_분당_병원.csv")

In [98]:
bundang.head()

,상세영업상태명,인허가일자,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,의료인수,입원실수,병상수,진료과목내용명,위도,경도
0,폐업,1995-04-10,2008-07-28,경기도 성남시 분당구 야탑동351,경기도 성남시 분당구 야탑로 59 (야탑동),분당차한방병원,한방병원,18,19,50,"한방내과, 한방부인과, 한방소아과, 한방안·이비인후·피부과, 한방신경정신과, 한방재...",37.41025,127.12525
1,폐업,2013-11-25,2023-07-21,NaN,"경기도 성남시 분당구 황새울로342번길 15, 5층일부, 6층 (서현동, 은성빌딩)",마디나은병원,병원,3,8,30,"정형외과, 신경외과, 마취통증의학과, 영상의학과",37.386423,127.123941
2,폐업,2010-03-15,2011-09-01,"경기도 성남시 분당구 서현동 253번지 2호 지하1층 일부,2층,5층","경기도 성남시 분당구 황새울로351번길 8 (서현동,지하1층 일부,2층,5층)",좋은이웃병원,병원,5,14,37,"내과, 소아청소년과, 이비인후과, 피부과, 영상의학과, 진단검사의학과",37.379383,127.113275
3,폐업,2012-05-04,2018-06-01,NaN,"경기도 성남시 분당구 미금일로90번길 32, 301~330호. 337~341호 (구...",순병원,병원,7,14,39,"내과, 신경과, 외과, 정형외과, 신경외과, 성형외과, 마취통증의학과, 소아청소년과...",37.34762,127.107229
4,폐업,2016-04-14,2023-06-15,NaN,"경기도 성남시 분당구 서현로 184, 405호, 406호 (서현동, 엘지분당에클라트)",송병재한방병원,한방병원,4,5,30,"재활의학과, 한방내과, 한방부인과, 한방소아과, 한방안·이비인후·피부과, 한방신경정...",37.388374,127.123789


### 필터링 기준 (의원)
> 사용 열 : "상세영업상태명", "폐업일자", "소재지전체주소", "도로명전체주소", "사업장명", "업태구분명", "의료인수", "입원실수", "병상수", "진료과목내용명"
- 필터링 (1) : 분당, 일산 단어가 포함된 위치 우선적으로 선택
- 전처리 (1) : 주소 기반 위도, 경도 관련 데이터 Search

In [3]:
## 병원
data_medical = pd.read_csv("./의원.csv", encoding='utf-8')
data_medical_sort = data_medical[["상세영업상태명", "인허가일자","폐업일자", "소재지전체주소", "도로명전체주소", "사업장명", "업태구분명", "의료인수", "입원실수", "병상수", "진료과목내용명"]]

In [4]:
data_medical_sort = data_medical_sort.copy()
data_medical_sort['위도'] = None
data_medical_sort['경도'] = None

In [74]:
ilsan = data_medical_sort[
    data_medical_sort["도로명전체주소"].combine_first(data_medical_sort["소재지전체주소"]).str.contains("고양시 일산", na=False)
].reset_index(drop=True)

ilsan.head()

,상세영업상태명,인허가일자,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,의료인수,입원실수,병상수,진료과목내용명,위도,경도
0,영업중,2019-03-06,NaN,경기도 고양시 일산동구 중산동 1667번지 8호,"경기도 고양시 일산동구 고봉로 266, 5층 (중산동)",미래마취통증의학과의원,의원,2.0,0.0,0.0,"정형외과, 신경외과, 마취통증의학과, 재활의학과",None,None
1,영업중,2019-03-08,NaN,NaN,"경기도 고양시 일산동구 정발산로 38, 장항동이스턴시티 207호 (장항동)",마음미소정신건강의학과의원,의원,1.0,0.0,0.0,정신건강의학과,None,None
2,영업중,2019-03-15,NaN,NaN,"경기도 고양시 일산동구 강송로 119, 용신코아 3층 302호 (백석동)",이재한의원,한의원,1.0,0.0,0.0,"한방내과, 한방부인과, 한방소아과, 한방안·이비인후·피부과, 한방신경정신과, 한방재...",None,None
3,영업중,2019-01-28,NaN,NaN,"경기도 고양시 일산동구 정발산로 38, 장항동이스턴시티 208호 일부, 408호 (...",정발산튼튼척의원,의원,3.0,0.0,0.0,"외과, 정형외과, 마취통증의학과, 재활의학과",None,None
4,영업중,2019-02-18,NaN,NaN,"경기도 고양시 일산동구 중산로 244, 고은프라자 3층 301호 (중산동)",열린부부한의원,한의원,1.0,0.0,0.0,"한방내과, 한방부인과, 한방소아과, 한방안·이비인후·피부과, 한방신경정신과, 한방재...",None,None


In [75]:
update_coordinates(ilsan, kakao_api_key)
# ilsan.to_csv("./dataset_life_infra_일산_병원.csv", encoding='utf-8')

 78%|███████▊  | 907/1161 [01:14<00:36,  6.97it/s]

Failed to get location info for: 경기도 고양시 일산동구 일산동1137-1. 2층 / nan


 96%|█████████▋| 1118/1161 [01:33<00:04,  9.41it/s]

Failed to get location info for: 경기도 고양시 일산동구 마두2동 뉴코아백화점일산점 901호 일부 / nan


100%|██████████| 1161/1161 [01:37<00:00, 11.95it/s]


In [ ]:
# API 상으로 검색 안되는 위치 -> 직접 입력
ilsan[ilsan['위도'].isna()]

,상세영업상태명,인허가일자,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,의료인수,입원실수,병상수,진료과목내용명,위도,경도
905,폐업,2002-05-08,2008-10-01,경기도 고양시 일산동구 일산동1137-1. 2층,NaN,진한의원,한의원,1.0,0.0,0.0,한방내과,None,None
1115,폐업,2011-04-20,2012-07-31,경기도 고양시 일산동구 마두2동 뉴코아백화점일산점 901호 일부,NaN,보메디의원,의원,1.0,0.0,0.0,"내과, 성형외과, 마취통증의학과, 소아청소년과, 피부과, 가정의학과",None,None


In [ ]:
# ilsan.to_csv("./dataset_life_infra_일산_의원.csv", encoding='utf-8')

In [15]:
bundang = data_medical_sort[
    data_medical_sort["도로명전체주소"].combine_first(data_medical_sort["소재지전체주소"]).str.contains("성남시 분당", na=False)
].reset_index(drop=True)

bundang.head()

,상세영업상태명,인허가일자,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,의료인수,입원실수,병상수,진료과목내용명,위도,경도
0,폐업,2008-09-18,2016-08-22,경기도 성남시분당구 금곡동 158번지 미도프라자 202호,"경기도 성남시 분당구 성남대로 168, 202호 (금곡동, 미도프라자)",리즈마취통증의학과의원,의원,1.0,0.0,0.0,"내과, 마취통증의학과, 피부과",None,None
1,폐업,2008-09-23,2013-09-02,"경기도 성남시분당구 수내동 83-3번지 돌마소매시장 301호일부, 304호","경기도 성남시 분당구 불정로 255, 301,304호 (수내동, 돌마소매시장)",미래정형외과의원,의원,1.0,0.0,0.0,"내과, 외과, 정형외과, 신경외과, 피부과, 비뇨의학과, 영상의학과, 재활의학과",None,None
2,폐업,2012-01-19,2015-05-11,경기도 성남시 분당구 662번지 판교타워 601호,"경기도 성남시 분당구 판교역로192번길 16, 601호 (삼평동, 판교타워)",S도쿄치과의원,치과의원,1.0,0.0,0.0,치과,None,None
3,폐업,2010-09-20,2017-04-01,경기도 성남시 분당구 수내동 63번지 1호 금산프라자 401호(일부),"경기도 성남시 분당구 내정로166번길 43, 401(일부)호 (수내동, 금산프라자)",씨씨엘의원,의원,1.0,0.0,0.0,"내과, 성형외과, 소아청소년과, 이비인후과, 피부과, 가정의학과",None,None
4,폐업,2004-07-19,2006-07-25,경기도 성남시 분당구 정자동 8번지 4호 태남프라자 508호,"경기도 성남시 분당구 정자일로 234, 508호 (정자동, 태남프라자)",S&H(Slim&Health)의원,의원,1.0,0.0,0.0,내과,None,None


In [16]:
update_coordinates(bundang, kakao_api_key)

 30%|██▉       | 492/1651 [01:56<04:10,  4.62it/s]

Failed to get location info for: 경기도 성남시분당구 야탑동 532번지 6호 정우스토아 201호 / 경기도 성남시 분당구 야탑남로 13, 201호 (야탑동, 정우스토아)


 34%|███▍      | 560/1651 [02:14<07:47,  2.33it/s]

Failed to get location info for: 경기도 성남시분당구 야탑동 149번지 10호 테크노프라자 401호 / 경기도 성남시 분당구 야탑남로 249, 401호 (야탑동, 테크노프라자)


100%|██████████| 1651/1651 [05:27<00:00,  5.04it/s]

Failed to get location info for: 경기도 성남시 분당구 정자동 정든마을 동아아파트 106-201 / nan


In [8]:
bundang.head()

,상세영업상태명,인허가일자,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,의료인수,입원실수,병상수,진료과목내용명,위도,경도
0,폐업,2008-09-18,2016-08-22,경기도 성남시분당구 금곡동 158번지 미도프라자 202호,"경기도 성남시 분당구 성남대로 168, 202호 (금곡동, 미도프라자)",리즈마취통증의학과의원,의원,1.0,0.0,0.0,"내과, 마취통증의학과, 피부과",37.350888,127.1095
1,폐업,2008-09-23,2013-09-02,"경기도 성남시분당구 수내동 83-3번지 돌마소매시장 301호일부, 304호","경기도 성남시 분당구 불정로 255, 301,304호 (수내동, 돌마소매시장)",미래정형외과의원,의원,1.0,0.0,0.0,"내과, 외과, 정형외과, 신경외과, 피부과, 비뇨의학과, 영상의학과, 재활의학과",37.366939,127.126393
2,폐업,2012-01-19,2015-05-11,경기도 성남시 분당구 662번지 판교타워 601호,"경기도 성남시 분당구 판교역로192번길 16, 601호 (삼평동, 판교타워)",S도쿄치과의원,치과의원,1.0,0.0,0.0,치과,37.397745,127.111433
3,폐업,2010-09-20,2017-04-01,경기도 성남시 분당구 수내동 63번지 1호 금산프라자 401호(일부),"경기도 성남시 분당구 내정로166번길 43, 401(일부)호 (수내동, 금산프라자)",씨씨엘의원,의원,1.0,0.0,0.0,"내과, 성형외과, 소아청소년과, 이비인후과, 피부과, 가정의학과",37.370357,127.122641
4,폐업,2004-07-19,2006-07-25,경기도 성남시 분당구 정자동 8번지 4호 태남프라자 508호,"경기도 성남시 분당구 정자일로 234, 508호 (정자동, 태남프라자)",S&H(Slim&Health)의원,의원,1.0,0.0,0.0,내과,37.371197,127.106283


In [17]:
bundang[bundang['위도'].isna()]

,상세영업상태명,인허가일자,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,의료인수,입원실수,병상수,진료과목내용명,위도,경도
491,폐업,2003-04-01,2010-06-11,경기도 성남시분당구 야탑동 532번지 6호 정우스토아 201호,"경기도 성남시 분당구 야탑남로 13, 201호 (야탑동, 정우스토아)",윤재호한의원 (휴업),한의원,1.0,0.0,0.0,"한방내과, 한방소아과, 한방안·이비인후·피부과",None,None
559,폐업,2003-02-04,2005-03-14,경기도 성남시분당구 야탑동 149번지 10호 테크노프라자 401호,"경기도 성남시 분당구 야탑남로 249, 401호 (야탑동, 테크노프라자)",메드플랜병리과의원,의원,1.0,0.0,0.0,"내과, 소아청소년과",None,None
1650,폐업,2000-06-19,2004-09-04,경기도 성남시 분당구 정자동 정든마을 동아아파트 106-201,NaN,소생한의원,한의원,1.0,0.0,0.0,"내과, 한방내과, 한방부인과, 한방소아과, 한방안·이비인후·피부과, 한방신경정신과,...",None,None


In [18]:
bundang.to_csv("./dataset_life_infra_분당_의원.csv")